In [19]:
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from myfunc import text_clean

In [3]:
# 기업 데이터가 없이 페이지 껍데기만 있는 주소
# url = "https://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage"
# r = requests.get(url)
# print(r.status_code)
# soup = bs(r.content, "lxml")
# soup

200


<!DOCTYPE html>
<html lang="ko">
<head>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="kr" http-equiv="content-language"/>
<meta content="text/css" http-equiv="content-style-type"/>
<meta content="no-cache" http-equiv="pragma"/>
<meta content="no-cache" http-equiv="cache-control"/>
<meta content="no" http-equiv="imagetoolbar"/>
<meta content="대한민국 대표 기업공시채널 KIND(c)2014" name="copyright"/>
<meta content="대한민국 대표 기업공시채널 KIND" name="description"/>
<meta content="대한민국 대표 기업공시채널 KIND" name="keywords"/>
<title>대한민국 대표 기업공시채널 KIND</title>
<link href="../images/kind.ico" rel="shortcut icon"/>
<link href="../js/jquery/themes-base/jquery-ui.min.css" media="all" rel="stylesheet" type="text/css"/>
<link href="../css/default_new.css" media="all" rel="stylesheet" type="text/css"/>
<link href="../css/common.css?version=20251219" media="all" rel="stylesheet" type="text/css"/>
<link href="../css/style.css?versio

In [20]:
# html 가져오기

url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
# & : [Edit]-Replace로 ', '우로 바꾸기
# 문자열은 "", 숫자는 그대로 두고, 값이 없는 건 제거
payload = dict(method="searchCorpList", pageIndex=1, currentPageSize=100, orderMode=3, orderStat="D", 
                searchType=13, fiscalYearEnd="all", location="all")

r = requests.get(url, params=payload)
print(r.url)
print(r.status_code)
soup = bs(r.content, "lxml")
soup

https://kind.krx.co.kr/corpgeneral/corpList.do?method=searchCorpList&pageIndex=1&currentPageSize=100&orderMode=3&orderStat=D&searchType=13&fiscalYearEnd=all&location=all
200


<html><body><section class="scrarea type-00">
<table class="list type-00 tmt30" summary="회사명, 업종, 주요제품, 상장일, 결산월, 대표자명, 홈페이지, 지역">
<caption>목록</caption>
<colgroup>
<col width="*"/>
<col width="15%"/>
<col width="20%"/>
<col width="11%"/>
<col width="8%"/>
<col width="10%"/>
<col width="8%"/>
<col width="12%"/>
</colgroup>
<thead>
<tr class="first" id="title-contents">
</tr>
</thead>
<tbody>
<tr class="first">
<td class="first" title="삼성기업인수목적13호(주)"><img alt="코스닥" class="vmiddle legend" src="/images/common/icn_t_ko.gif"/> <a href="#companysum" id="companysum" onclick="companysummary_open('0115H'); return false;" title="삼성스팩13호"> 삼성스팩13호</a> </td>
<td class="textOverflow" title="금융 지원 서비스업">금융 지원 서비스업</td>
<td class="textOverflow" title="기업인수합병">기업인수합병</td>
<td class="txc">2026-01-21</td>
<td class="txc">12월</td>
<td class="txc" title="성상환">성상환</td>
<td class="txc">
</td>
<td class="txc">서울특별시</td>
</tr>
<tr>
<td class="first" title="(주)엔비알모션"><img alt="코스닥" class="vmiddle legend" src="

In [10]:
# 컬럼명 수집하기

col_names = soup.select_one("table.list.type-00.tmt30")["summary"].split(", ")
col_names

['회사명', '업종', '주요제품', '상장일', '결산월', '대표자명', '홈페이지', '지역']

In [33]:
all_result = []
for row in soup.select("table.list.type-00.tmt30 tbody tr")[:1]:
    result = {}
    for idx , (key, tds) in enumerate(zip(col_names, row.select("td"))):
        if idx == 0:
            market = tds.select_one("img")['alt']
            code = tds.select_one("a")["onclick"].split("'")[1]+"0"
            company_name = tds.get_text(strip=True)
            result["종류"] = market
            result["종목코드"] = code
            result["회사명"] = company_name
#             print(market, code, company_name)
        elif key == '홈페이지':
            print(tds)
            print(tds.select_one("a"))
        else:
            result.setdefault(key, tds.get_text(strip=True))
#     print(result)
    all_result.append(result)
        
# result.setdefault(key, []).append(tds.get_text(strip=True))
# df = pd.DateFrame(result)
# df

<td class="txc">
</td>
None
